# Linear Algebra in OSCAR

* Authors: Martin Bies, Aaruni Kaushik
* Version: OSCAR version 1.2.2 or newer.

## Introduction

This tutorial provides an introduction to linear algebra in *OSCAR*. It highlights key functionalities through examples, making it easier for users to get started. However, this tutorial does not cover all available functionality. Instead, we focus on a selection of essential functions and features. For a more comprehensive guide, see the official documentation [here](https://docs.oscar-system.org/stable/LinearAlgebra/intro/).

No prior knowledge of *OSCAR* is required for this tutorial, but a basic understanding of Julia is assumed.

## Contents

The content of this tutorial is as follows:

1. [Creating Matrices](#1-Creating-Matrices)  
2. [Solving Linear Equation Systems](#2-Solving-Linear-Equation-Systems)  
3. [Eigenvalues and Eigenspaces](#3-Eigenvalues-and-Eigenspaces)  
4. [Normal Forms](#4-Normal-Forms)  
5. [Vectors](#5-Vectors)  
6. [Matrix Spaces](#6-Matrix-Spaces)  


In [1]:
using Oscar

## 1 Creating Matrices

*OSCAR* has its own type of matrix, independent of Julia's in-built matrices. This is due to technical limitations in Julia. Specifically, every matrix in *OSCAR* has a unique coefficient ring assigned to it. For a Julia matrix, this is not possible. Also, types (even with parameters) are not suitable for this purpose. For details, see https://docs.oscar-system.org/stable/AbstractAlgebra/matrix/#Basic-matrix-functionality. For conversion of the "normal" julia matrices into *OSCAR* matrices, see for instance https://docs.oscar-system.org/stable/AbstractAlgebra/matrix/#Conversion-to-Julia-matrices,-iteration-and-broacasting.

*Oscar* matrices are typically created with the (*Oscar*) function `matrix` with a small "m"; whereas Julia's built-in matrices have types written `Matrix{...}` with a big "M". In conclusion, use `matrix` with small "m" within *OSCAR*.

There are many ways to create matrices in *OSCAR*. We will now demonstrate a few. Let us emphasize again, that in the *OSCAR* matrix constructors, we must specify the coefficient ring.

In [2]:
m = matrix(ZZ, [1 2 3 ; 4 5 6])

[1   2   3]
[4   5   6]

Note that the above secretly creates a Julia matrix and then turns it into an *OSCAR* matrix over the ring $ZZ$. This is the julia-ish way of creating such matrices.

However, in practical applications, you might find this syntax not very human-readable. Eyeball the following:

In [3]:
R, x = polynomial_ring(ZZ, :x)
matrix(R, [x^2 -x-1; x^2-x -1])

[    x^2   -x - 1]
[x^2 - x       -1]

Here are alternatives to create the above matrix $m$, but such that the syntax is readable even in involved settings such as polynomial entries:

In [4]:
m = matrix(ZZ, [[1,2,3], [4,5,6]])
m = matrix(ZZ, 2,3, [1,2,3,4,5,6])

[1   2   3]
[4   5   6]

We can create a matrix with the same coefficients, but defined over the field of rational numbers:

In [5]:
m2 = matrix(QQ, [1 2 3; 4 5 6])

[1   2   3]
[4   5   6]

Note that these two matrices print the same but they have different types.

In [6]:
typeof(m)

ZZMatrix

In [7]:
typeof(m2)

QQMatrix

Despite these matrices being defined over different coefficient rings, we can (for instance) add them:

In [8]:
m+m2

[2    4    6]
[8   10   12]

In [9]:
base_ring(ans)

Rational field

You will likely not use the function base_ring too often. Still, it can be rather useful in order to identify over which ring a matrix is defined, for instance when (trying to) add or compare two matrices.

In the case at hand, just as addition was possible above, so is comparison:

In [10]:
m == m2

true

This fails for other coefficient rings. As an example, let us consider the following matrices:

In [11]:
F9, a = finite_field(9, "a")
I_f9 = identity_matrix(F9, 4)

[1   0   0   0]
[0   1   0   0]
[0   0   1   0]
[0   0   0   1]

In [12]:
I_zz = identity_matrix(ZZ, 4)

[1   0   0   0]
[0   1   0   0]
[0   0   1   0]
[0   0   0   1]

In [13]:
I_qq = identity_matrix(QQ, 4)

[1   0   0   0]
[0   1   0   0]
[0   0   1   0]
[0   0   0   1]

Note that we used a convenience function to create these identity matrices. More such functionality does exist, see e.g. https://docs.oscar-system.org/stable/AbstractAlgebra/matrix/#Basic-matrix-functionality. Here is another example use of such a convenience function:

In [14]:
zero_matrix(ZZ, 4, 5)

[0   0   0   0   0]
[0   0   0   0   0]
[0   0   0   0   0]
[0   0   0   0   0]

Let us now return to our matrices $I_{zz}$, $I_{qq}$ and $I_{f9}$. Those are defined over different coefficient rings. While they print the same, not all pairs of them can be compared:

In [15]:
I_zz == I_qq

true

In general, elements of different rings may not be compared nor combined. This extends to matrices defined over different coefficients rings.

However, if there is a unique embedding of one ring into the other, then *OSCAR* will do this for us automatically. Thereby, sometimes matrices over different rings can be compared, and sometimes not. In the case at hand, $I_{f9}$ and $I_{qq}$ cannot be compared, but $I_{zz}$ and $I_{f9}$ can:

In [16]:
I_zz == I_f9

true

## 2 Solving Linear Equation Systems

Let us solve the following system of linear equations in $\texttt{OSCAR}$ over $\mathbb{Q}$:

$$ \left[ \begin{array}{ccc} 1 & 2 & 3 \\ 2 & 4 & 6 \end{array} \right] \cdot v = \left[ \begin{array}{c} -5 \\ -10 \end{array} \right] $$

In [17]:
m = matrix(QQ, [[1,2,3], [2,4,6]])

[1   2   3]
[2   4   6]

In [18]:
b = matrix(QQ, [[-5], [-10]])

[ -5]
[-10]

In [19]:
solve(m, b; side = :right)

[-5]
[ 0]
[ 0]

Side comment: The semicolon separates the mandatory arguments in the beginning from optional keyword argument.

But of course, this is merely a single solution. To obtain all solutions, we need to add the nullspace.

In [20]:
d, nb = nullspace(m)

(2, [-2 -3; 1 0; 0 1])

The first return value is the dimension of the nullspace, and the second is a basis of the nullspace.

In [21]:
nb

[-2   -3]
[ 1    0]
[ 0    1]

Additional information is available in the *OSCAR* documentation. See for instance:
* https://docs.oscar-system.org/stable/AbstractAlgebra/linear_solving/#solving_chapter.
* https://docs.oscar-system.org/stable/AbstractAlgebra/matrix/#nullspace-Union{Tuple{MatElem{T}},%20Tuple{T}}%20where%20T%3C:FieldElem

## 3 Eigenvalues and Eigenspaces

In this section, we demonstrate how to compute eigenvalues and eigenspaces.

Consider the following two matrices over the integers resp. rationals.

In [22]:
mQ = matrix(QQ, [1//2 0; 0 2])

[1//2   0]
[   0   2]

In [23]:
mZ = matrix(ZZ, [0 1; -1 0])

[ 0   1]
[-1   0]

The eigenvalues are zeros of the characteristic polynomial:

In [24]:
characteristic_polynomial(mQ)

x^2 - 5//2*x + 1

In [25]:
characteristic_polynomial(mZ)

x^2 + 1

Certainly, we can ask directly ask for the eigenvalues. A first attempt might look like this:

In [26]:
eigenvalues(mQ)

2-element Vector{QQFieldElem}:
 2
 1//2

In [27]:
eigenvalues(mZ)

ZZRingElem[]

The second list is empty because by default eigenvalues are searched only in the `base_ring` of the matrix, and the matrix has no integer eigenvalues.

We can also ask for eigenvalues in a larger ring, e.g. in the algebraic closure of the rationals:

In [28]:
K = algebraic_closure(QQ)

Algebraic closure of rational field

In [29]:
lambda1, lambda2 = eigenvalues(K, mZ)

2-element Vector{QQBarFieldElem}:
 Root 1.00000*im of x^2 + 1
 Root -1.00000*im of x^2 + 1

Next we may wish to compute eigenspaces. Again we can first try a single argument version:

In [30]:
eigenspaces(mQ)

Dict{QQFieldElem, QQMatrix} with 2 entries:
  1//2 => [1 0]
  2    => [0 1]

By design, issuing `eigenspaces(mZ)` raises an error: "Please specify the field over which to compute (since matrix is not over a field)".

This is because `OSCAR` currently only supports `eigenspaces` which are vector spaces, so over a field. But as before, if we specify nothing else it performs computations over the base ring of the matrix, which here is the ring of integers.

Once again we can resolve this by specifying a field over which to compute the eigenspaces, e.g. again the algebraic closure of the rationals:

In [31]:
eigenspaces(K, mZ)

Dict{QQBarFieldElem, AbstractAlgebra.Generic.MatSpaceElem{QQBarFieldElem}} with 2 entries:
  Root 1.00000*im of x^2 + 1  => [Root 1.00000*im of x^2 + 1 Root 1.00000 of x - 1]
  Root -1.00000*im of x^2 + 1 => [Root -1.00000*im of x^2 + 1 Root 1.00000 of x - 1]

Of course we can also ask for a specific eigenspace corresponding to a given eigenvalue. Here we don't need to specify a field if the eigenvalue is already a field element.

In [32]:
eigenspace(mZ, lambda1)

[Root 1.00000*im of x^2 + 1   Root 1.00000 of x - 1]

In [33]:
eigenspace(mQ, 2)

[0   1]

### Caveats

The result of `eigenspace` is a matrix whose **rows are a basis** for the eigenspace. This basis need not be orthogonal.

Similarly `eigenspaces` returns a `Dict` associating to each eigenvalue a matrix whose rows form a basis.

Not surprisingly we get the following when the given value is not an eigenvalue:

In [34]:
eigenspace(mQ, 3)

0 by 2 empty matrix

## 4 Normal Forms

There are many many normal forms, we demonstrate how to compute the Smith and the Hermite normal form.

### 4.1 Smith Normal Form

In [35]:
M = matrix(ZZ, [[-16, -28, -19], [42, 69, 46], [-42, -72, -49]])

[-16   -28   -19]
[ 42    69    46]
[-42   -72   -49]

In [36]:
base_ring(M)

Integer ring

In [37]:
snf(M)

[1   0    0]
[0   1    0]
[0   0   30]

In [38]:
S, T, U = snf_with_transform(M)

([1 0 0; 0 1 0; 0 0 30], [13 -3 -8; -18 4 11; -69 16 43], [-2 -4 19; 1 3 -16; 0 -1 6])

In [39]:
T * M * U == S

true

 For more details see https://docs.oscar-system.org/stable/AbstractAlgebra/matrix/#Smith-normal-form.

### 4.2 Hermite Normal Form

Similarly to the above, we can also compute the Hermite normal form (HNF). Let us emphasize that *OSCAR* computes the **row** HNF.

In [40]:
M

[-16   -28   -19]
[ 42    69    46]
[-42   -72   -49]

In [41]:
base_ring(M)

Integer ring

In [42]:
hnf(M)

[2   2   4]
[0   3   3]
[0   0   5]

In [43]:
H, U = hnf_with_transform(M)

([2 2 4; 0 3 3; 0 0 5], [13 -2 -7; 0 -1 -1; 21 -4 -12])

In [44]:
U * M == H

true

For more details see https://docs.oscar-system.org/stable/AbstractAlgebra/matrix/#Hermite-normal-form.

## 5 Vectors

One way of creating vectors, is by extracting rows and columns of a matrix. Let us demonstrate this:

In [45]:
M = matrix(QQ, [[1,2,3], [4,5,6]])

[1   2   3]
[4   5   6]

In [46]:
row = M[1:1,:]

[1   2   3]

In [47]:
col = M[:,1:1]

[1]
[4]

Certainly, we can multiply the original matrix from the right with the transpose of row:

In [48]:
M * transpose(row)

[14]
[32]

Likewise, we can multiply the original matrix from the left with the transpose of col:

In [49]:
transpose(col) * M

[17   22   27]

Here is an alternative way of extracting the first row, so that you can multiply it (secretly being treated as a column vector) from the right with the matrix $M$, but without having to use the method "transpose":

In [50]:
v = M[1,:]

3-element Vector{QQFieldElem}:
 1
 2
 3

In [51]:
M*v

2-element Vector{QQFieldElem}:
 14
 32

Do not be confused. While we extract the first row of our matrix $M$, julia prints it as a column, simply because that is the default for any list of elements (the data structure of which is called "Vector" in julia, to add to the confusion). Still, we extracted the first row of the matrix.

# 6 Matrix Spaces

Fix non-negative integers $r$, $c$ and a ring $R$. The set of all $r \times c$ matrices defined over the coefficient ring $R$ is called a matrix space. *OSCAR* provides functionality for these spaces. For details, the interested reader may for instance wish to consult https://docs.oscar-system.org/stable/AbstractAlgebra/matrix_spaces/#Basic-matrix-space-functionality.

Let us give an example for how to obtain the matrix space of a matrix:

In [52]:
M

[1   2   3]
[4   5   6]

In [53]:
base_ring(M)

Rational field

In [54]:
parent(M)

Matrix space of 2 rows and 3 columns
  over rational field

As you can see, the matrix space records the base ring/coefficient ring, as well as the dimensions of the matrices it contains. The elements of a matrix space are called `MatElem` or `MatrixElem` -- you may see these names in documentation about *OSCAR* functions operating on matrices.